# Materials and Emission Streams

This is a continuation of the Multiscale MILP example

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Multiscale MILP

In [ ]:
# !pip install energiapy # uncomment and run to install Energia, if not in environment
from energia import *

m = Model('design_scheduling_material')
m.q = Periods()
m.y = 4 * m.q

In [ ]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.usd.spend(m.wf.capacity) == 990637 + 3354
m.usd.spend(m.wf.operate) == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.usd.spend(m.pv.capacity) == 567000 + 872046
m.usd.spend(m.pv.operate) == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.usd.spend(m.lii.capacity) == 1302182 + 41432
m.usd.spend(m.lii.inventory) == 2000

⚖   Initiated solar balance in (l0, q)                                      ⏱ 0.0002 s
🔗  Bound [≤] solar consume in (l0, q)                                       ⏱ 0.0010 s
⚖   Initiated wind balance in (l0, y)                                       ⏱ 0.0001 s
🔗  Bound [≤] wind consume in (l0, y)                                        ⏱ 0.0007 s
⚖   Initiated power balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≥] power release in (l0, q)                                       ⏱ 0.0016 s
🔗  Bound [≤] wf capacity in (l0, y)                                         ⏱ 0.0003 s
🔗  Bound [≥] wf capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≤] wf operate in (l0, q)                                          ⏱ 0.0004 s
🔗  Bound [=] usd spend in (l0, y)                                           ⏱ 0.0003 s
🧭  Mapped time for operate (wf, l0, q) ⟺ (wf, l0, y)                        ⏱ 0.0004 s
🔗  Bound [=] usd spend in (l0, y)          

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [17]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [ ]:
m.gwp.emit(m.wf.capacity) == 1000
m.gwp.emit(m.pv.capacity) == 2000
m.gwp.emit(m.lii.capacity) == 3000
m.emit.show()

🔗  Bound [=] gwp emit in (l0, y)                                            ⏱ 0.0004 s
🔗  Bound [=] gwp emit in (l0, y)                                            ⏱ 0.0004 s
🔗  Bound [=] gwp emit in (l0, y)                                            ⏱ 0.0004 s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [ ]:
m.cement = Material()
m.cement.consume <= 1000000
m.usd.spend(m.cement.consume) == 17
m.gwp.emit(m.cement.consume) == 0.9

⚖   Initiated cement balance in (l0, y)                                     ⏱ 0.0001 s
🔗  Bound [≤] cement consume in (l0, y)                                      ⏱ 0.0010 s
🔗  Bound [=] usd spend in (l0, y)                                           ⏱ 0.0005 s
🔗  Bound [=] gwp emit in (l0, y)                                            ⏱ 0.0007 s


Next, we provide details of the use of cement across all operations 

In [ ]:
m.cement.use(m.wf.capacity) == 400
m.cement.use(m.pv.capacity) == 560
m.cement.use(m.lii.capacity) == 300

⚖   Updated cement balance with use(cement, l0, y, capacity, wf)            ⏱ 0.0001 s
🔗  Bound [=] cement use in (l0, y)                                          ⏱ 0.0013 s
⚖   Updated cement balance with use(cement, l0, y, capacity, pv)            ⏱ 0.0001 s
🔗  Bound [=] cement use in (l0, y)                                          ⏱ 0.0015 s
⚖   Updated cement balance with use(cement, l0, y, invcapacity, lii.stored) ⏱ 0.0001 s
🔗  Bound [=] cement use in (l0, y)                                          ⏱ 0.0014 s


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [21]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [22]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [23]:
m.network.locate(m.wf, m.pv, m.lii)

💡  Assumed wf capacity unbounded in (l0, y)                                 ⏱ 0.0001 s
💡  Assumed wf operate bounded by capacity in (l0, q)                        ⏱ 0.0001 s
⚖   Updated power balance with produce(power, l0, q, operate, wf)           ⏱ 0.0002 s
🔗  Bound [=] power produce in (l0, q)                                       ⏱ 0.0013 s
⚖   Updated wind balance with expend(wind, l0, y, operate, wf)              ⏱ 0.0001 s
🔗  Bound [=] wind expend in (l0, y)                                         ⏱ 0.0011 s
🏭  Operating streams introduced for wf in l0                                ⏱ 0.0040 s
🏗   Construction streams introduced for wf in l0                            ⏱ 0.0000 s
🌍  Located wf in l0                                                         ⏱ 0.0062 s
💡  Assumed pv capacity unbounded in (l0, y)                                 ⏱ 0.0001 s
💡  Assumed pv operate bounded by capacity in (l0, q)                        ⏱ 0.0001 s
⚖   Updated power balance with produce(powe

## Optimization 

### Minimize GWP
Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [24]:
m.gwp.emit.opt()

🧭  Mapped samples for emit (gwp, l0, y, capacity, wf) ⟺ (gwp, l0, y)        ⏱ 0.0004 s
🧭  Mapped samples for emit (gwp, l0, y, capacity, pv) ⟺ (gwp, l0, y)        ⏱ 0.0003 s
🧭  Mapped samples for emit (gwp, l0, y, invcapacity, lii.stored) ⟺ (gwp, l0, y) ⏱ 0.0003 s
🧭  Mapped samples for emit (gwp, l0, y, consume, cement) ⟺ (gwp, l0, y)     ⏱ 0.0003 s
📝  Generated Program(design_scheduling_material).mps                        ⏱ 0.0059 s


Read MPS format model from file Program(design_scheduling_material).mps
Reading time = 0.00 seconds
PROGRAM(DESIGN_SCHEDULING_MATERIAL): 93 rows, 87 columns, 215 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0084 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 93 rows, 87 columns and 215 nonzeros
Model fingerprint: 0x347f9574
Variable types: 84 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+06]
Presolve removed 88 rows and 81 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 18 nonzeros
Variable types: 6 continuous, 0 integer (0 binary)

Root relaxation: objective 4.752148e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    475214.81481 475214.815  0.00% 

📝  Generated Solution object for Program(design_scheduling_material). See .solution ⏱ 0.0006 s
✅  Program(design_scheduling_material) optimized using gurobi. Display using .output() ⏱ 0.0258 s


### Minimize Cost
The same problem can also be minimized in terms of cost

In [25]:
m.usd.spend.opt()

🧭  Mapped samples for spend (usd, l0, y, capacity, wf) ⟺ (usd, l0, y)       ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, operate, wf) ⟺ (usd, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, capacity, pv) ⟺ (usd, l0, y)       ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, operate, pv) ⟺ (usd, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, invcapacity, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, inventory, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, consume, cement) ⟺ (usd, l0, y)    ⏱ 0.0002 s
📝  Generated Program(design_scheduling_material).mps                        ⏱ 0.0077 s


Read MPS format model from file Program(design_scheduling_material).mps
Reading time = 0.00 seconds
PROGRAM(DESIGN_SCHEDULING_MATERIAL): 94 rows, 88 columns, 223 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0107 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 94 rows, 88 columns and 223 nonzeros
Model fingerprint: 0x59224a16
Variable types: 85 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+06]
Presolve removed 82 rows and 76 columns
Presolve time: 0.00s
Presolved: 12 rows, 12 columns, 34 nonzeros
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 3.024203e+08, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.024203e+08 3.0242e+08  0.0

📝  Generated Solution object for Program(design_scheduling_material). See .solution ⏱ 0.0006 s
✅  Program(design_scheduling_material) optimized using gurobi. Display using .output() ⏱ 0.0274 s


In [26]:
m.show()

# Mathematical Program for Program(design_scheduling_material)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [27]:
m.output()

# Solution for Program(design_scheduling_material)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>